<a href="https://colab.research.google.com/github/t1seo/AIFFEL_Hackerthon-1/blob/main/taewon/model_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/data/Restaurant_Visitor_Forecasting

/content/drive/MyDrive/data/Restaurant_Visitor_Forecasting


In [3]:
import numpy as np
import pandas as pd
from sklearn import ensemble, neighbors, linear_model, metrics, preprocessing
from datetime import datetime

## 데이터 불러오기

In [4]:
data = {
    'tra': pd.read_csv('air_visit_data.csv'),
    'as': pd.read_csv('air_store_info.csv'),
    'hs': pd.read_csv('hpg_store_info.csv'),
    'ar': pd.read_csv('air_reserve.csv'),
    'hr': pd.read_csv('hpg_reserve.csv'),
    'id': pd.read_csv('store_id_relation.csv'),
    'tes': pd.read_csv('sample_submission.csv'),
    'hol': pd.read_csv('date_info.csv').rename(columns={'calendar_date':'visit_date'})
}

# 데이터 전처리

### hpg 데이터 중에서 store_id_relation에 있는 id만 사용한다

`hpg` 데이터에는 타깃 데이터인 `visitors`가 없다. (`visitors`는 `air_visit_data`에 있다)

키 값이 없는 `hpg` 데이터는 과감히 쳐내준다.

#### ===코멘트 수정 필요====

In [5]:
# len(data['as'])

In [6]:
# air_visits에 있는 식당ID가 모두 air_store_info에 있는가? 
# data['tra']['air_store_id'].isin(data['as']['air_store_id']).all()

In [7]:
data['id'] # id는 총 150개

,air_store_id,hpg_store_id
0,air_63b13c56b7201bd9,hpg_4bc649e72e2a239a
1,air_a24bf50c3e90d583,hpg_c34b496d0305a809
2,air_c7f78b4f3cba33ff,hpg_cd8ae0d9bbd58ff9
3,air_947eb2cae4f3e8f2,hpg_de24ea49dc25d6b8
4,air_965b2e0cf4119003,hpg_653238a84804d8e7
...,...,...
145,air_b8925441167c3152,hpg_0cc0abae3a8b52de
146,air_f8233ad00755c35c,hpg_098e4dd30e54fee6
147,air_6607fe3671242ce3,hpg_1004356dc8ab69ec
148,air_ccd19a5bc5573ae5,hpg_df22bf146d2c7e19


- `id`는 총 150개이고 `hpg` 데이터 중에서 `id`에 있는 데이터들만 사용을 한다.

In [8]:
len(data['hr'])

2000320

In [9]:
# hpg_reserve.csv에 store_id_relation.csv를 hpg_store_id 기준으로 inner merge
data['hr'] = pd.merge(data['hr'], data['id'], how='inner', on=['hpg_store_id'])

data['hr'].head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors,air_store_id
0,hpg_878cc70b1abc76f7,2016-01-01 19:00:00,2016-01-01 15:00:00,4,air_db80363d35f10926
1,hpg_878cc70b1abc76f7,2016-01-02 19:00:00,2016-01-02 14:00:00,2,air_db80363d35f10926
2,hpg_878cc70b1abc76f7,2016-01-03 18:00:00,2016-01-02 20:00:00,6,air_db80363d35f10926
3,hpg_878cc70b1abc76f7,2016-01-06 20:00:00,2016-01-04 22:00:00,3,air_db80363d35f10926
4,hpg_878cc70b1abc76f7,2016-01-11 18:00:00,2016-01-11 14:00:00,2,air_db80363d35f10926


In [10]:
len(data['hr'])

28183

- `id`만 있는 데이터들만 `merge`한 결과 데이터 개수가 크게 줄었다. 
    - 2000320 -> 28183

## 시계열 데이터 처리
- 연월일시 -> 연월일
- `reserve_datetime_diff` 컬럼 생성(예약한 시간 - 방문 예정 시간)

---
*reserve_datetime_diff* 상관관계

In [11]:
for df in ['ar', 'hr']:
    data[df]['visit_datetime'] = pd.to_datetime(data[df]['visit_datetime'])
      # str을 datetime으로 변환
    data[df]['visit_datetime'] = data[df]['visit_datetime'].dt.date
      # datetime을 dt.date로 변환 
    data[df]['reserve_datetime'] = pd.to_datetime(data[df]['reserve_datetime'])
      # str을 datetime으로 변환
    data[df]['reserve_datetime'] = data[df]['reserve_datetime'].dt.date
      # datetime을 dt.date로 변환 
    data[df]['reserve_datetime_diff'] = data[df].apply(lambda r: (r['visit_datetime'] - r['reserve_datetime']).days, axis=1)
      # difference time을 의미하는 변수 생성 (예약한 시간 - 실제 방문 시간)
    # data[df] = data[df].groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].sum()
    # data[df] = data[df].rename(columns={'visit_datetime':'visit_date'})
      # air_reserve와 hpg_ reserve로 group을 나눔
# print(data[df].head())

In [12]:
data['ar'].head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors,reserve_datetime_diff
0,air_877f79706adbfb06,2016-01-01,2016-01-01,1,0
1,air_db4b38ebe7a7ceff,2016-01-01,2016-01-01,3,0
2,air_db4b38ebe7a7ceff,2016-01-01,2016-01-01,6,0
3,air_877f79706adbfb06,2016-01-01,2016-01-01,2,0
4,air_db80363d35f10926,2016-01-01,2016-01-01,5,0


In [13]:
data['hr'].head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors,air_store_id,reserve_datetime_diff
0,hpg_878cc70b1abc76f7,2016-01-01,2016-01-01,4,air_db80363d35f10926,0
1,hpg_878cc70b1abc76f7,2016-01-02,2016-01-02,2,air_db80363d35f10926,0
2,hpg_878cc70b1abc76f7,2016-01-03,2016-01-02,6,air_db80363d35f10926,1
3,hpg_878cc70b1abc76f7,2016-01-06,2016-01-04,3,air_db80363d35f10926,2
4,hpg_878cc70b1abc76f7,2016-01-11,2016-01-11,2,air_db80363d35f10926,0


### `air_store_id`와 `visit_datetime`으로 `groupby`

In [14]:
for df in ['ar', 'hr']:
    data[df] = data[df].groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].sum()
    data[df] = data[df].rename(columns={'visit_datetime':'visit_date'})

In [15]:
data['ar'].head()

,air_store_id,visit_date,reserve_datetime_diff,reserve_visitors
0,air_00a91d42b08b08d9,2016-10-31,0,2
1,air_00a91d42b08b08d9,2016-12-05,4,9
2,air_00a91d42b08b08d9,2016-12-14,6,18
3,air_00a91d42b08b08d9,2016-12-17,6,2
4,air_00a91d42b08b08d9,2016-12-20,2,4


In [16]:
data['hr'].head()

,air_store_id,visit_date,reserve_datetime_diff,reserve_visitors
0,air_00a91d42b08b08d9,2016-01-14,3,2
1,air_00a91d42b08b08d9,2016-01-15,6,4
2,air_00a91d42b08b08d9,2016-01-16,3,2
3,air_00a91d42b08b08d9,2016-01-22,3,2
4,air_00a91d42b08b08d9,2016-01-29,6,5


- `visit_datetime`에서 날짜만 뽑아서 새로운 컬럼을 만들어준다.
- `lambda` 함수를 이용해 방문 예정 시간 `visit_datetime`과 예약한 시간 `reserve_datetime`을 뺴줘 새로운 컬럼인 `reserve_datetime_diff`를 만들어준다.


## training data (`air_visit_data`) feature 추가

In [17]:
data['tra']

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6
...,...,...,...
252103,air_24e8414b9b07decb,2017-04-18,6
252104,air_24e8414b9b07decb,2017-04-19,6
252105,air_24e8414b9b07decb,2017-04-20,7
252106,air_24e8414b9b07decb,2017-04-21,8


### 요일(dow), 연(year), 월(month)를 의미하는 컬럼 추가

In [18]:
# air_visit_data

data['tra']['visit_date'] = pd.to_datetime(data['tra']['visit_date'])
  # str을 datetime으로 변환
data['tra']['dow'] = data['tra']['visit_date'].dt.dayofweek
  # 요일을 의미하는 숫자 변수 생성
data['tra']['year'] = data['tra']['visit_date'].dt.year
  # 년도 year 변수 생성
data['tra']['month'] = data['tra']['visit_date'].dt.month
  # 달 month 변수 생성
data['tra']['visit_date'] = data['tra']['visit_date'].dt.date
  # datetime을 dt.date로 변환
data['tra'].head()

,air_store_id,visit_date,visitors,dow,year,month
0,air_ba937bf13d40fb24,2016-01-13,25,2,2016,1
1,air_ba937bf13d40fb24,2016-01-14,32,3,2016,1
2,air_ba937bf13d40fb24,2016-01-15,29,4,2016,1
3,air_ba937bf13d40fb24,2016-01-16,22,5,2016,1
4,air_ba937bf13d40fb24,2016-01-18,6,0,2016,1


## 테스트 데이터 셋에 컬럼 추가

In [19]:
data['tes']

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,0
1,air_00a91d42b08b08d9_2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,0
3,air_00a91d42b08b08d9_2017-04-26,0
4,air_00a91d42b08b08d9_2017-04-27,0
...,...,...
32014,air_fff68b929994bfbd_2017-05-27,0
32015,air_fff68b929994bfbd_2017-05-28,0
32016,air_fff68b929994bfbd_2017-05-29,0
32017,air_fff68b929994bfbd_2017-05-30,0


### 테스트 셋 컬럼 추가
- 예측해야 하는 날
- `air_store_id`
- 요일, 연도, 달(month)

In [20]:
# sample_submission

data['tes']['visit_date'] = data['tes']['id'].map(lambda x: str(x).split('_')[2])
  # id 변수에서 날짜를 뽑아와 visit_date 변수 생성
data['tes']['air_store_id'] = data['tes']['id'].map(lambda x: '_'.join(x.split('_')[:2]))
  # id 변수에서 store id를 뽑아와 air_store_id 변수 생성
data['tes']['visit_date'] = pd.to_datetime(data['tes']['visit_date'])
  # str을 datetime으로 변환
data['tes']['dow'] = data['tes']['visit_date'].dt.dayofweek
  # 요일을 의미하는 숫자 변수 생성
data['tes']['year'] = data['tes']['visit_date'].dt.year
  # 년도 year 변수 생성
data['tes']['month'] = data['tes']['visit_date'].dt.month
  # 달 month 변수 생성
data['tes']['visit_date'] = data['tes']['visit_date'].dt.date
  # datetime을 dt.date로 변환

data['tes'].head()

,id,visitors,visit_date,air_store_id,dow,year,month
0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,air_00a91d42b08b08d9,6,2017,4
1,air_00a91d42b08b08d9_2017-04-24,0,2017-04-24,air_00a91d42b08b08d9,0,2017,4
2,air_00a91d42b08b08d9_2017-04-25,0,2017-04-25,air_00a91d42b08b08d9,1,2017,4
3,air_00a91d42b08b08d9_2017-04-26,0,2017-04-26,air_00a91d42b08b08d9,2,2017,4
4,air_00a91d42b08b08d9_2017-04-27,0,2017-04-27,air_00a91d42b08b08d9,3,2017,4


### 테스트 셋 821개의 식당 -> x7  = 5747
- 요일 7개
- 821 x 7 = 5747개

In [21]:
# test 데이터에서 식당 목록만 뽑아낸다
unique_stores = data['tes']['air_store_id'].unique()
print(len(unique_stores))

821


- `test` 셋에는 총 821개의 식당이 있다.

In [22]:
# unique_stores = data['tes']['air_store_id'].unique() # air_store id만을 뽑아낸다.

# 일주일에 해당하는 만큼 unique_stores x 7
stores = pd.concat([pd.DataFrame({'air_store_id': unique_stores, 
                                  'dow': [i] * len(unique_stores)}) for i in range(7)], 
                   axis=0, 
                   ignore_index=True).reset_index(drop=True)

print(len(stores))

5747


In [23]:
stores.tail(7)

,air_store_id,dow
5740,air_fe22ef5a9cbef123,6
5741,air_fe58c074ec1445ea,6
5742,air_fea5dc9594450608,6
5743,air_fee8dcf4d619598e,6
5744,air_fef9ccb3ba0da2f7,6
5745,air_ffcc2d5087e1b476,6
5746,air_fff68b929994bfbd,6


## 가게의 '요일별' `visitors`의 산술 통계량 계산
- min
- mean
- median
- max
- count ??

In [24]:
# 산술 통계량을 뽑아내서 feature로 train data에 추가해준다 -> 자료

tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].min().rename(columns={'visitors':'min_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].mean().rename(columns={'visitors':'mean_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].median().rename(columns={'visitors':'median_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].max().rename(columns={'visitors':'max_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].count().rename(columns={'visitors':'count_observations'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 

stores.head()

,air_store_id,dow,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations
0,air_00a91d42b08b08d9,0,1.0,22.457143,19.0,47.0,35.0
1,air_0164b9927d20bcc3,0,2.0,7.500000,6.0,19.0,20.0
2,air_0241aa3964b7f861,0,2.0,8.920635,8.0,23.0,63.0
3,air_0328696196e46f18,0,2.0,6.416667,4.0,27.0,12.0
4,air_034a3d5b40d5b1b1,0,1.0,11.864865,10.0,66.0,37.0


In [25]:
print(len(stores))

5747


## 요일별 통계량에 `air_store_info` merge

In [26]:
# air store를 stores에 merge
stores = pd.merge(stores, data['as'], how='left', on=['air_store_id']) 
stores.head()

,air_store_id,dow,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,air_genre_name,air_area_name,latitude,longitude
0,air_00a91d42b08b08d9,0,1.0,22.457143,19.0,47.0,35.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595
1,air_0164b9927d20bcc3,0,2.0,7.500000,6.0,19.0,20.0,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
2,air_0241aa3964b7f861,0,2.0,8.920635,8.0,23.0,63.0,Izakaya,Tōkyō-to Taitō-ku Higashiueno,35.712607,139.779996
3,air_0328696196e46f18,0,2.0,6.416667,4.0,27.0,12.0,Dining bar,Ōsaka-fu Ōsaka-shi Nakanochō,34.701279,135.528090
4,air_034a3d5b40d5b1b1,0,1.0,11.864865,10.0,66.0,37.0,Cafe/Sweets,Ōsaka-fu Ōsaka-shi Ōhiraki,34.692337,135.472229


In [27]:
print(len(stores))

5747


## 업종과 식당 위치를 라벨 인코딩

In [28]:
lbl = preprocessing.LabelEncoder()
  # labelEncoding 진행 
stores['air_genre_name'] = lbl.fit_transform(stores['air_genre_name'])
stores['air_area_name'] = lbl.fit_transform(stores['air_area_name'])

stores.head()

,air_store_id,dow,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,air_genre_name,air_area_name,latitude,longitude
0,air_00a91d42b08b08d9,0,1.0,22.457143,19.0,47.0,35.0,6,44,35.694003,139.753595
1,air_0164b9927d20bcc3,0,2.0,7.500000,6.0,19.0,20.0,6,62,35.658068,139.751599
2,air_0241aa3964b7f861,0,2.0,8.920635,8.0,23.0,63.0,7,82,35.712607,139.779996
3,air_0328696196e46f18,0,2.0,6.416667,4.0,27.0,12.0,4,98,34.701279,135.528090
4,air_034a3d5b40d5b1b1,0,1.0,11.864865,10.0,66.0,37.0,2,102,34.692337,135.472229


## 연휴 데이터 전처리

In [29]:
data['hol']

,visit_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0
...,...,...,...
512,2017-05-27,Saturday,0
513,2017-05-28,Sunday,0
514,2017-05-29,Monday,0
515,2017-05-30,Tuesday,0


### 요일을 라벨 인코딩

In [30]:
data['hol']['visit_date'] = pd.to_datetime(data['hol']['visit_date'])
data['hol']['day_of_week'] = lbl.fit_transform(data['hol']['day_of_week'])
data['hol']['visit_date'] = data['hol']['visit_date'].dt.date
data['hol']

,visit_date,day_of_week,holiday_flg
0,2016-01-01,0,1
1,2016-01-02,2,1
2,2016-01-03,3,1
3,2016-01-04,1,0
4,2016-01-05,5,0
...,...,...,...
512,2017-05-27,2,0
513,2017-05-28,3,0
514,2017-05-29,1,0
515,2017-05-30,5,0


## train 데이터에 holiday 데이터 merge

In [31]:
  # holiday의 날짜 변수 가공한뒤 merge
train = pd.merge(data['tra'], data['hol'], how='left', on=['visit_date']) 

train.head()

,air_store_id,visit_date,visitors,dow,year,month,day_of_week,holiday_flg
0,air_ba937bf13d40fb24,2016-01-13,25,2,2016,1,6,0
1,air_ba937bf13d40fb24,2016-01-14,32,3,2016,1,4,0
2,air_ba937bf13d40fb24,2016-01-15,29,4,2016,1,0,0
3,air_ba937bf13d40fb24,2016-01-16,22,5,2016,1,2,0
4,air_ba937bf13d40fb24,2016-01-18,6,0,2016,1,1,0


## test 데이터에 holiday 데이터 merge

In [32]:
test = pd.merge(data['tes'], data['hol'], how='left', on=['visit_date'])
test.head()

,id,visitors,visit_date,air_store_id,dow,year,month,day_of_week,holiday_flg
0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,air_00a91d42b08b08d9,6,2017,4,3,0
1,air_00a91d42b08b08d9_2017-04-24,0,2017-04-24,air_00a91d42b08b08d9,0,2017,4,1,0
2,air_00a91d42b08b08d9_2017-04-25,0,2017-04-25,air_00a91d42b08b08d9,1,2017,4,5,0
3,air_00a91d42b08b08d9_2017-04-26,0,2017-04-26,air_00a91d42b08b08d9,2,2017,4,6,0
4,air_00a91d42b08b08d9_2017-04-27,0,2017-04-27,air_00a91d42b08b08d9,3,2017,4,4,0


## train 데이터에 stores 데이터(요일별 데이터) merge

In [33]:
train = pd.merge(data['tra'], stores, how='left', on=['air_store_id','dow']) 
train.head()

,air_store_id,visit_date,visitors,dow,year,month,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,air_genre_name,air_area_name,latitude,longitude
0,air_ba937bf13d40fb24,2016-01-13,25,2,2016,1,7.0,23.843750,25.0,57.0,64.0,4.0,62.0,35.658068,139.751599
1,air_ba937bf13d40fb24,2016-01-14,32,3,2016,1,2.0,20.292308,21.0,54.0,65.0,4.0,62.0,35.658068,139.751599
2,air_ba937bf13d40fb24,2016-01-15,29,4,2016,1,4.0,34.738462,35.0,61.0,65.0,4.0,62.0,35.658068,139.751599
3,air_ba937bf13d40fb24,2016-01-16,22,5,2016,1,6.0,27.651515,27.0,53.0,66.0,4.0,62.0,35.658068,139.751599
4,air_ba937bf13d40fb24,2016-01-18,6,0,2016,1,2.0,13.754386,12.0,34.0,57.0,4.0,62.0,35.658068,139.751599


## test 데이터에 stores 데이터(요일별 데이터) merge

In [34]:
test = pd.merge(data['tes'], stores, how='left', on=['air_store_id','dow'])
test.head()

,id,visitors,visit_date,air_store_id,dow,year,month,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,air_genre_name,air_area_name,latitude,longitude
0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,air_00a91d42b08b08d9,6,2017,4,2.0,2.000000,2.0,2.0,1.0,6,44,35.694003,139.753595
1,air_00a91d42b08b08d9_2017-04-24,0,2017-04-24,air_00a91d42b08b08d9,0,2017,4,1.0,22.457143,19.0,47.0,35.0,6,44,35.694003,139.753595
2,air_00a91d42b08b08d9_2017-04-25,0,2017-04-25,air_00a91d42b08b08d9,1,2017,4,1.0,24.350000,24.5,43.0,40.0,6,44,35.694003,139.753595
3,air_00a91d42b08b08d9_2017-04-26,0,2017-04-26,air_00a91d42b08b08d9,2,2017,4,15.0,28.125000,28.0,52.0,40.0,6,44,35.694003,139.753595
4,air_00a91d42b08b08d9_2017-04-27,0,2017-04-27,air_00a91d42b08b08d9,3,2017,4,15.0,29.868421,30.0,47.0,38.0,6,44,35.694003,139.753595


## train과 test 데이터에 air 데이터, hpg 데이터 merge

In [35]:
train.head()

,air_store_id,visit_date,visitors,dow,year,month,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,air_genre_name,air_area_name,latitude,longitude
0,air_ba937bf13d40fb24,2016-01-13,25,2,2016,1,7.0,23.843750,25.0,57.0,64.0,4.0,62.0,35.658068,139.751599
1,air_ba937bf13d40fb24,2016-01-14,32,3,2016,1,2.0,20.292308,21.0,54.0,65.0,4.0,62.0,35.658068,139.751599
2,air_ba937bf13d40fb24,2016-01-15,29,4,2016,1,4.0,34.738462,35.0,61.0,65.0,4.0,62.0,35.658068,139.751599
3,air_ba937bf13d40fb24,2016-01-16,22,5,2016,1,6.0,27.651515,27.0,53.0,66.0,4.0,62.0,35.658068,139.751599
4,air_ba937bf13d40fb24,2016-01-18,6,0,2016,1,2.0,13.754386,12.0,34.0,57.0,4.0,62.0,35.658068,139.751599


In [36]:
test.head()

,id,visitors,visit_date,air_store_id,dow,year,month,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,air_genre_name,air_area_name,latitude,longitude
0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,air_00a91d42b08b08d9,6,2017,4,2.0,2.000000,2.0,2.0,1.0,6,44,35.694003,139.753595
1,air_00a91d42b08b08d9_2017-04-24,0,2017-04-24,air_00a91d42b08b08d9,0,2017,4,1.0,22.457143,19.0,47.0,35.0,6,44,35.694003,139.753595
2,air_00a91d42b08b08d9_2017-04-25,0,2017-04-25,air_00a91d42b08b08d9,1,2017,4,1.0,24.350000,24.5,43.0,40.0,6,44,35.694003,139.753595
3,air_00a91d42b08b08d9_2017-04-26,0,2017-04-26,air_00a91d42b08b08d9,2,2017,4,15.0,28.125000,28.0,52.0,40.0,6,44,35.694003,139.753595
4,air_00a91d42b08b08d9_2017-04-27,0,2017-04-27,air_00a91d42b08b08d9,3,2017,4,15.0,29.868421,30.0,47.0,38.0,6,44,35.694003,139.753595


In [37]:
for df in ['ar', 'hr']:
    train = pd.merge(train, data[df], how='left', on=['air_store_id','visit_date']) 
    test = pd.merge(test, data[df], how='left', on=['air_store_id','visit_date'])

In [38]:
train.head()

,air_store_id,visit_date,visitors,dow,year,month,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,air_genre_name,air_area_name,latitude,longitude,reserve_datetime_diff_x,reserve_visitors_x,reserve_datetime_diff_y,reserve_visitors_y
0,air_ba937bf13d40fb24,2016-01-13,25,2,2016,1,7.0,23.843750,25.0,57.0,64.0,4.0,62.0,35.658068,139.751599,NaN,NaN,NaN,NaN
1,air_ba937bf13d40fb24,2016-01-14,32,3,2016,1,2.0,20.292308,21.0,54.0,65.0,4.0,62.0,35.658068,139.751599,NaN,NaN,NaN,NaN
2,air_ba937bf13d40fb24,2016-01-15,29,4,2016,1,4.0,34.738462,35.0,61.0,65.0,4.0,62.0,35.658068,139.751599,NaN,NaN,NaN,NaN
3,air_ba937bf13d40fb24,2016-01-16,22,5,2016,1,6.0,27.651515,27.0,53.0,66.0,4.0,62.0,35.658068,139.751599,NaN,NaN,NaN,NaN
4,air_ba937bf13d40fb24,2016-01-18,6,0,2016,1,2.0,13.754386,12.0,34.0,57.0,4.0,62.0,35.658068,139.751599,NaN,NaN,NaN,NaN


In [39]:
test.head()

,id,visitors,visit_date,air_store_id,dow,year,month,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,air_genre_name,air_area_name,latitude,longitude,reserve_datetime_diff_x,reserve_visitors_x,reserve_datetime_diff_y,reserve_visitors_y
0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,air_00a91d42b08b08d9,6,2017,4,2.0,2.000000,2.0,2.0,1.0,6,44,35.694003,139.753595,NaN,NaN,NaN,NaN
1,air_00a91d42b08b08d9_2017-04-24,0,2017-04-24,air_00a91d42b08b08d9,0,2017,4,1.0,22.457143,19.0,47.0,35.0,6,44,35.694003,139.753595,NaN,NaN,NaN,NaN
2,air_00a91d42b08b08d9_2017-04-25,0,2017-04-25,air_00a91d42b08b08d9,1,2017,4,1.0,24.350000,24.5,43.0,40.0,6,44,35.694003,139.753595,NaN,NaN,NaN,NaN
3,air_00a91d42b08b08d9_2017-04-26,0,2017-04-26,air_00a91d42b08b08d9,2,2017,4,15.0,28.125000,28.0,52.0,40.0,6,44,35.694003,139.753595,NaN,NaN,NaN,NaN
4,air_00a91d42b08b08d9_2017-04-27,0,2017-04-27,air_00a91d42b08b08d9,3,2017,4,15.0,29.868421,30.0,47.0,38.0,6,44,35.694003,139.753595,NaN,NaN,NaN,NaN


## train과 test 데이터에서 reserve관련 데이터 NaN 값들을 -1로 채워준다

이 값들은 예약을 하지 않고 온 손님들을 의미한다.

In [40]:
print(train.describe())
print(train.head())

            visitors  ...  reserve_visitors_y
count  252108.000000  ...        13550.000000
mean       20.973761  ...            6.803469
std        16.757007  ...            7.686077
min         1.000000  ...            1.000000
25%         9.000000  ...            2.000000
50%        17.000000  ...            4.000000
75%        29.000000  ...            8.000000
max       877.000000  ...          157.000000

[8 rows x 17 columns]
           air_store_id  ... reserve_visitors_y
0  air_ba937bf13d40fb24  ...                NaN
1  air_ba937bf13d40fb24  ...                NaN
2  air_ba937bf13d40fb24  ...                NaN
3  air_ba937bf13d40fb24  ...                NaN
4  air_ba937bf13d40fb24  ...                NaN

[5 rows x 19 columns]


In [41]:
col = [c for c in train if c not in ['id', 'air_store_id','visit_date','visitors']]
train = train.fillna(-1)
test = test.fillna(-1)

In [42]:
train.head()

,air_store_id,visit_date,visitors,dow,year,month,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,air_genre_name,air_area_name,latitude,longitude,reserve_datetime_diff_x,reserve_visitors_x,reserve_datetime_diff_y,reserve_visitors_y
0,air_ba937bf13d40fb24,2016-01-13,25,2,2016,1,7.0,23.843750,25.0,57.0,64.0,4.0,62.0,35.658068,139.751599,-1.0,-1.0,-1.0,-1.0
1,air_ba937bf13d40fb24,2016-01-14,32,3,2016,1,2.0,20.292308,21.0,54.0,65.0,4.0,62.0,35.658068,139.751599,-1.0,-1.0,-1.0,-1.0
2,air_ba937bf13d40fb24,2016-01-15,29,4,2016,1,4.0,34.738462,35.0,61.0,65.0,4.0,62.0,35.658068,139.751599,-1.0,-1.0,-1.0,-1.0
3,air_ba937bf13d40fb24,2016-01-16,22,5,2016,1,6.0,27.651515,27.0,53.0,66.0,4.0,62.0,35.658068,139.751599,-1.0,-1.0,-1.0,-1.0
4,air_ba937bf13d40fb24,2016-01-18,6,0,2016,1,2.0,13.754386,12.0,34.0,57.0,4.0,62.0,35.658068,139.751599,-1.0,-1.0,-1.0,-1.0


## 학습 및 예측

In [43]:
def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred)**0.5

In [44]:
lr = linear_model.LinearRegression(n_jobs=-1)
etc = ensemble.ExtraTreesRegressor(n_estimators=225, max_depth=5, n_jobs=-1, random_state=3)
knn = neighbors.KNeighborsRegressor(n_jobs=-1, n_neighbors=4)

lr.fit(train[col], np.log1p(train['visitors'].values))
etc.fit(train[col], np.log1p(train['visitors'].values))
knn.fit(train[col], np.log1p(train['visitors'].values))

print('RMSE LinearRegressor: ', RMSLE(np.log1p(train['visitors'].values), lr.predict(train[col])))
print('RMSE ExtraTreesRegressor: ', RMSLE(np.log1p(train['visitors'].values), etc.predict(train[col])))
print('RMSE KNNRegressor: ', RMSLE(np.log1p(train['visitors'].values), knn.predict(train[col])))

test['visitors'] = (etc.predict(test[col]) / 2) +(knn.predict(test[col]) / 2) # ensemble
test['visitors'] = np.expm1(test['visitors']).clip(lower=0.)
test[['id','visitors']].to_csv('lr_submission.csv', index=False, float_format='%.2f')

RMSE LinearRegressor:  0.5559363280886771
RMSE ExtraTreesRegressor:  0.5238810527950593
RMSE KNNRegressor:  0.4255759709458109
